# 01 Data preparation

Processing xlsx files from data folder into suitable inputs and generate other input files

In [1]:
import pandas as pd
import numpy as np
import os
import pickle

In [2]:
data_dir = "DATA"
info_dir = data_dir
out_dir = "01_outputs_2020"

#os.makedirs(info_dir, exist_ok = True)
os.makedirs(out_dir, exist_ok = True)

## Modules

Load data about modules and drugs.

In [3]:
### DATA
### remove whitespaces in names (modules), remove duplicates

modules_df = pd.read_excel(
    os.path.join(info_dir, "ALL_DATA.xlsx"), sheet_name = "modules", index_col = 0)

# display(modules_df)

selected_modules = modules_df.index.tolist()
print(len(selected_modules), ' - Size after reading')

# remove duplicates
selected_modules = modules_df.index.unique().tolist()
print(len(selected_modules), ' - Size after remove duplicates')

# remove whitespaces in modules' names 
selected_modules = [d.strip() for d in selected_modules]

print('Selected modules list: ', len(selected_modules), selected_modules)


10  - Size after reading
10  - Size after remove duplicates
Selected modules list:  10 ['CDK1/2', 'CDK4/6', 'IKK', 'TLR4', 'mTOR', 'PAK', 'ERK', 'JAK/STAT', 'PLK1', 'ITK']


In [4]:
### DATA
### remove whitespaces in names (modules, drugs), remove duplicates
### check the dimensions of the indicator IC50 1 uM = 1000 nM
### copy-paste as values, numbers, no formulas

IC50_df = pd.read_excel(
    os.path.join(info_dir, "ALL_DATA.xlsx"), sheet_name = "IC50s")

print(len(IC50_df.index), ' - Size after reading')
# display(IC50_df)

# rename
IC50_df = IC50_df.rename(columns = {"IC50, uM": "IC50"})

# manually correcting value IC50, Example  for IOX2 -> 30 nM  
#IC50_df.loc[IC50_df.index == 'IOX2', IC50_df.columns == 'IC50'] = 30/1000

# remove non-selected modules, modules' names with whitespaces or empty
IC50_df = IC50_df[IC50_df.Module.isin(selected_modules)]

print(len(IC50_df.index), ' - Size after remove modules')
# display(IC50_df)

# remove duplicates 
# Considering certain columns is optional. 
# Indexes, including time indexes are ignored.
IC50_df = IC50_df.drop_duplicates()

print(len(IC50_df.index), ' - Size after remove duplicates')
display(IC50_df)

20  - Size after reading
20  - Size after remove modules
20  - Size after remove duplicates


Module          Drug   IC50  Gamma
0     CDK1/2    dinaciclib  0.004      0
1     CDK1/2     CGP-60474  0.010      0
2     CDK4/6   palbociclib  0.020      0
3        IKK   BMS-345541   4.000      0
4        IKK        TPCA-1  0.100      0
5       TLR4    resatorvid  0.002      0
6       mTOR   palomid-529  0.050      0
7       mTOR  temsirolimus  1.760      0
8       mTOR    KU-0063794  0.030      0
9       mTOR      AZD-8055  0.020      0
10       PAK   PF-03758309  0.002      0
11       ERK    PD-0325901  0.015      0
12       ERK      TAK-733   0.005      0
13       ERK       RAF-265  5.000      0
14       ERK        AZ-628  3.000      0
15       ERK     FR-180204  3.100      0
16  JAK/STAT   tofacitinib  0.070      0
17  JAK/STAT   baricitinib  0.050      0
18      PLK1    volasertib  0.010      0
19       ITK    BMS-509744  0.050      0

In [5]:
modules = IC50_df.Module.unique().tolist()

print('IC50_df  modules list: ', len(modules), modules)
print()
print('Selected modules list: ', len(selected_modules), selected_modules)

### CHECK
print()
print('CHECK: ', len(selected_modules),'=?', len(modules))

n_modules = len(modules)


IC50_df  modules list:  10 ['CDK1/2', 'CDK4/6', 'IKK', 'TLR4', 'mTOR', 'PAK', 'ERK', 'JAK/STAT', 'PLK1', 'ITK']

Selected modules list:  10 ['CDK1/2', 'CDK4/6', 'IKK', 'TLR4', 'mTOR', 'PAK', 'ERK', 'JAK/STAT', 'PLK1', 'ITK']

CHECK:  10 =? 10


In [6]:
drugs = IC50_df.Drug.tolist()
print(len(drugs), ' - Size after reading')

# remove duplicates
drugs = IC50_df.Drug.unique().tolist()
print(len(drugs), ' - Size after remove duplicates')

# remove whitespaces in drugs' names (necessary for some)
drugs = [d.strip() for d in drugs]

# remove duplicates after remove whitespaces
drugs = list(set(drugs))
print(len(drugs), ' - Size after remove duplicates without whitespaces')

print('Drugs list: ', len(drugs), drugs)

n_drugs = len(drugs)

20  - Size after reading
20  - Size after remove duplicates
20  - Size after remove duplicates without whitespaces
Drugs list:  20 ['AZ-628', 'palbociclib', 'BMS-509744', 'PF-03758309', 'KU-0063794', 'CGP-60474', 'TAK-733', 'baricitinib', 'TPCA-1', 'RAF-265', 'AZD-8055', 'palomid-529', 'temsirolimus', 'PD-0325901', 'resatorvid', 'FR-180204', 'BMS-345541', 'dinaciclib', 'tofacitinib', 'volasertib']


## L1000 meta data

Get sig_id for selected drugs.

In [7]:
sig_info_df = pd.read_csv(os.path.join(data_dir, "siginfo_beta.txt"),sep='\t', index_col = 25)
#sig_info_df_HUVEC = sig_info_df.loc[sig_info_df['cell_mfc_name'].str.contains('HUVEC')]
sig_info_df = sig_info_df.loc[sig_info_df['cell_mfc_name'].str.contains('THP1')]

#LFC_PLCg = pd.read_csv(os.path.join(data_dir, "PLCg_Data_log_2020.csv"),index_col=0)
#sig_info_df_PLCg = sig_info_df.loc[LFC_PLCg.index]
#sig_info_df = pd.concat([sig_info_df_HUVEC,sig_info_df_PLCg],axis=0)

display(sig_info_df)

/tmp/ipykernel_26210/50942704.py:1: DtypeWarning: Columns (0,3,4,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  sig_info_df = pd.read_csv(os.path.join(data_dir, "siginfo_beta.txt"),sep='\t', index_col = 25)


bead_batch  nearest_dose  \
sig_id                                                                 
KMS002_THP1_6H:BRD-K69885235-001-01-2:0             b5        0.0001   
KMS002_THP1_24H:BRD-K03073752-001-02-5:0.12         b5        0.1200   
KMS002_THP1_6H:BRD-K51556300-001-01-9:0.38          b5        0.3700   
KMS002_THP1_24H:BRD-K46625679-001-01-3:1.11         b5        1.1100   
KMS002_THP1_6H:J23                                  b5           NaN   
...                                                ...           ...   
AICHI002_THP1_4H:G14                               b39           NaN   
AICHI001_THP1_4H:G16                               b39        0.1250   
AICHI001_THP1_4H:K13                               b39       10.0000   
AICHI001_THP1_4H:A13                               b39       10.0000   
AICHI001_THP1_24H:B05                              b32           NaN   

                                             pert_dose pert_dose_unit  \
sig_id                                                                  
KMS002_THP1_6H:BRD-K69885235-001-01-2:0            NaN             uM   
KMS002_THP1_24H:BRD-K03073752-001-02-5:0.12   0.120000             uM   
KMS002_THP1_6H:BRD-K51556300-001-01-9:0.38    0.380000             uM   
KMS002_THP1_24H:BRD-K46625679-001-01-3:1.11   1.110000             uM   
KMS002_THP1_6H:J23                                 NaN            NaN   
...                                                ...            ...   
AICHI002_THP1_4H:G14                          2.499810             uM   
AICHI001_THP1_4H:G16                          0.156238             uM   
AICHI001_THP1_4H:K13                         10.000000             uM   
AICHI001_THP1_4H:A13                         10.000000             uM   
AICHI001_THP1_24H:B05                              NaN            NaN   

                                            pert_idose pert_itime  pert_time  \
sig_id                                                                         
KMS002_THP1_6H:BRD-K69885235-001-01-2:0            NaN        6 h        6.0   
KMS002_THP1_24H:BRD-K03073752-001-02-5:0.12    0.12 uM       24 h       24.0   
KMS002_THP1_6H:BRD-K51556300-001-01-9:0.38     0.37 uM        6 h        6.0   
KMS002_THP1_24H:BRD-K46625679-001-01-3:1.11    1.11 uM       24 h       24.0   
KMS002_THP1_6H:J23                                 NaN        6 h        6.0   
...                                                ...        ...        ...   
AICHI002_THP1_4H:G14                            2.5 uM        4 h        4.0   
AICHI001_THP1_4H:G16                          0.125 uM        4 h        4.0   
AICHI001_THP1_4H:K13                             10 uM        4 h        4.0   
AICHI001_THP1_4H:A13                             10 uM        4 h        4.0   
AICHI001_THP1_24H:B05                              NaN       24 h       24.0   

                                            pert_time_unit cell_mfc_name  \
sig_id                                                                     
KMS002_THP1_6H:BRD-K69885235-001-01-2:0                  h          THP1   
KMS002_THP1_24H:BRD-K03073752-001-02-5:0.12              h          THP1   
KMS002_THP1_6H:BRD-K51556300-001-01-9:0.38               h          THP1   
KMS002_THP1_24H:BRD-K46625679-001-01-3:1.11              h          THP1   
KMS002_THP1_6H:J23                                       h          THP1   
...                                                    ...           ...   
AICHI002_THP1_4H:G14                                     h          THP1   
AICHI001_THP1_4H:G16                                     h          THP1   
AICHI001_THP1_4H:K13                                     h          THP1   
AICHI001_THP1_4H:A13                                     h          THP1   
AICHI001_THP1_24H:B05                                    h          THP1   

                                               pert_mfc_id  ...  cell_iname  \
sig_id                                          

In [8]:
# now filtering so only the required drugs are present
sig_info_df = sig_info_df.loc[sig_info_df['cmap_name'].isin(drugs)]

# here's what we have now
display(sig_info_df)

bead_batch  nearest_dose  pert_dose pert_dose_unit  \
sig_id                                                                     
CPC006_THP1_6H:C04          f2b4         10.00  10.000000             uM   
CPC006_THP1_6H:G06          f2b4         10.00  10.000000             uM   
CPC006_THP1_6H:K09          f2b4         10.00  10.000000             uM   
CPC006_THP1_6H:M21          f2b4         10.00  10.000000             uM   
EMU001_THP1_6H:B19           b39         10.00  10.000000             uM   
...                          ...           ...        ...            ...   
AICHI001_THP1_4H:O14         b39          2.50   2.500000             uM   
AICHI001_THP1_24H:M18        b32          0.01   0.009766             uM   
AICHI001_THP1_4H:M13         b39         10.00  10.000000             uM   
AICHI001_THP1_4H:E24         b39          0.01   0.009766             uM   
AICHI001_THP1_4H:O17         b39          0.04   0.039062             uM   

                      pert_idose pert_itime  pert_time pert_time_unit  \
sig_id                                                                  
CPC006_THP1_6H:C04         10 uM        6 h        6.0              h   
CPC006_THP1_6H:G06         10 uM        6 h        6.0              h   
CPC006_THP1_6H:K09         10 uM        6 h        6.0              h   
CPC006_THP1_6H:M21         10 uM        6 h        6.0              h   
EMU001_THP1_6H:B19         10 uM        6 h        6.0              h   
...                          ...        ...        ...            ...   
AICHI001_THP1_4H:O14      2.5 uM        4 h        4.0              h   
AICHI001_THP1_24H:M18    0.01 uM       24 h       24.0              h   
AICHI001_THP1_4H:M13       10 uM        4 h        4.0              h   
AICHI001_THP1_4H:E24     0.01 uM        4 h        4.0              h   
AICHI001_THP1_4H:O17     0.04 uM        4 h        4.0              h   

                      cell_mfc_name             pert_mfc_id  ...  cell_iname  \
sig_id                                                       ...               
CPC006_THP1_6H:C04             THP1  BRD-K69932463-001-03-1  ...        THP1   
CPC006_THP1_6H:G06             THP1  BRD-K67566344-001-01-8  ...        THP1   
CPC006_THP1_6H:K09             THP1  BRD-K51575138-001-01-7  ...        THP1   
CPC006_THP1_6H:M21             THP1  BRD-A62025033-001-01-8  ...        THP1   
EMU001_THP1_6H:B19             THP1           BRD-A50998626  ...        THP1   
...                             ...                     ...  ...         ...   
AICHI001_THP1_4H:O14           THP1           BRD-K13566078  ...        THP1   
AICHI001_THP1_24H:M18          THP1           BRD-K05804044  ...        THP1   
AICHI001_THP1_4H:M13           THP1           BRD-K05804044  ...        THP1   
AICHI001_THP1_4H:E24           THP1           BRD-K53581288  ...        THP1   
AICHI001_THP1_4H:O17           THP1           BRD-K13566078  ...        THP1   

                       det_wells  \
sig_id                             
CPC006_THP1_6H:C04           C04   
CPC006_THP1_6H:G06           G06   
CPC006_THP1_6H:K09           K09   
CPC006_THP1_6H:M21           M21   
EMU001_THP1_6H:B19           B19   
...                          ...   
AICHI001_THP1_4H:O14         O14   
AICHI001_THP1_24H:M18        M18   
AICHI001_THP1_4H:M13         M13   
AICHI001_THP1_4H:E24         E24   
AICHI001_THP1_4H:O17         O17   

                                                              det_plates  \
sig_id                                                                     
CPC006_THP1_6H:C04                    CPC006_THP1_6H_X1_F2B4_DUO52HI53LO   
CPC006_THP1_6H:G06     CPC006_THP1_6H_X1_F2B4_DUO52HI53LO|CPC006_THP1...   
CPC006_THP1_6H:K09                    CPC006_THP1_6H_X1_F2B4_DUO52HI53LO   
CPC006_THP1_6H:M21     CPC006_THP1_6H_X1_F2B4_DUO52HI53LO|CPC006_THP1...   
EMU001_THP1_6H:B19     EMU001_THP1_6H_X1_B39|EMU001_THP1_6H_X2_B39|EM...   
...                                      

In [9]:
exp_ids = sig_info_df.index.unique().tolist()
print('Experiments ids list: ', len(exp_ids))

n_experiments = len(exp_ids)

Experiments ids list:  269


Confirm data by checking the drugs of interest against the filtered L1000 meta data.

In [10]:
print(f"Number of drugs of interest:\t{len(drugs)}")
#print(f'Number of drugs in L1000 data:\t{len(sig_info_df.value_counts("drugs"))}')

#sig_info_df.value_counts("drugs")

Number of drugs of interest:	20


## L1000 data

In [11]:
LFC_df = pd.read_csv(os.path.join(data_dir, "level3_norm_thp1.csv"), index_col = 0)
display(LFC_df)

AARS     ABCB6     ABCC5     ABCF1  \
AICHI001_THP1_24H:A03              -0.051469  0.068087  1.619786  0.197440   
AICHI001_THP1_24H:A04               0.246263 -0.002630  0.004236  0.201474   
AICHI001_THP1_24H:A05               0.151863  0.053370 -0.194680  0.324075   
AICHI001_THP1_24H:A06               0.395531  0.381554 -0.213914  0.484775   
AICHI001_THP1_24H:A07              -1.160836  0.706071 -0.137480  0.157474   
...                                      ...       ...       ...       ...   
REP.B028_THP1_24H:P23              -0.096095  0.140660  0.049450  0.002725   
REP.B028_THP1_24H:P24              -0.007245  0.056210  0.040750 -0.164925   
ZTO.XPR001_THP1_408H:CMAP-000:-666 -0.076504  0.196899  0.026406  0.037738   
ZTO.XPR001_THP1_408H:GC1:-666       0.037214 -0.062505 -0.061845 -0.149995   
ZTO.XPR001_THP1_408H:GC2:-666       0.039291 -0.134394  0.035439  0.112257   

                                       ABCF3     ABHD4     ABHD6      ABL1  \
AICHI001_THP1_24H:A03              -0.092832  0.153559 -0.038607 -0.127486   
AICHI001_THP1_24H:A04               0.218100  0.132609  0.067376  0.083881   
AICHI001_THP1_24H:A05               0.011184  0.095559  0.120693  0.076748   
AICHI001_THP1_24H:A06               0.246634  0.167292  0.041659  0.219648   
AICHI001_THP1_24H:A07              -0.599216  1.100126 -0.208540  0.081448   
...                                      ...       ...       ...       ...   
REP.B028_THP1_24H:P23               0.037976  0.128351 -0.053827  0.008464   
REP.B028_THP1_24H:P24               0.172701 -0.147549 -0.077528  0.158815   
ZTO.XPR001_THP1_408H:CMAP-000:-666  0.163917  0.059711 -0.011489 -0.003306   
ZTO.XPR001_THP1_408H:GC1:-666      -0.120290 -0.078459  0.031846  0.302690   
ZTO.XPR001_THP1_408H:GC2:-666      -0.043627  0.018749 -0.020357 -0.299383   

                                       ACAA1     ACAT2  ...     ZMIZ1  \
AICHI001_THP1_24H:A03              -0.262796 -0.001191  ...  0.145774   
AICHI001_THP1_24H:A04              -0.212229 -0.285673  ...  0.063340   
AICHI001_THP1_24H:A05               0.304621 -0.146906  ...  0.121440   
AICHI001_THP1_24H:A06              -0.042629 -0.450957  ... -0.114177   
AICHI001_THP1_24H:A07              -0.165980 -0.827257  ...  0.821839   
...                                      ...       ...  ...       ...   
REP.B028_THP1_24H:P23               0.231246  0.220580  ...  0.007838   
REP.B028_THP1_24H:P24              -0.563929 -0.173469  ...  0.084538   
ZTO.XPR001_THP1_408H:CMAP-000:-666 -0.133241 -0.112347  ...  0.170960   
ZTO.XPR001_THP1_408H:GC1:-666       0.182437  0.132301  ... -0.028359   
ZTO.XPR001_THP1_408H:GC2:-666      -0.049196 -0.019953  ... -0.142601   

                                       ZMYM2    ZNF131    ZNF274    ZNF318  \
AICHI001_THP1_24H:A03              -0.699832  0.197302 -0.197423  0.170148   
AICHI001_THP1_24H:A04               0.223952 -0.145247  0.004727  0.238414   
AICHI001_THP1_24H:A05               0.088602 -0.174197 -0.245941 -0.143568   
AICHI001_THP1_24H:A06               0.187351  0.178503 -0.025890  0.529898   
AICHI001_THP1_24H:A07               0.503701  0.498320  0.071528 -0.117135   
...                                      ...       ...       ...       ...   
REP.B028_THP1_24H:P23              -0.461756  0.024425  0.164083  0.326541   
REP.B028_THP1_24H:P24               0.323094 -0.170676 -0.308566  0.312516   
ZTO.XPR001_THP1_408H:CMAP-000:-666  0.041612  0.040709 -0.096339  0.093352   
ZTO.XPR001_THP1_408H:GC1:-666      -0.019649 -0.099940  0.055280 -0.037169   
ZTO.XPR001_THP1_408H:GC2:-666      -0.021963  0.059231  0.041058 -0.056184   

                                      ZNF395    ZNF451    ZNF586    ZNF589  \
AICHI001_THP1_24H:A03               0.018956 -0.160991  0.160636  0.434180   
AICHI001_THP1_24H:A04               0.387924  0.082741  0.227103  0.365113   
AICHI001_THP1_24H:A05               0.195724 -0.084459 -0.009331  0.376179   
AICHI001_THP1_24H:A06               0.15

In [12]:
LFC_df = LFC_df[LFC_df.index.isin(exp_ids)]

#LFC_df = pd.concat([LFC_df,LFC_PLCg],axis=0)

# arrange experiments in same order as in list
LFC_df["sort_col"] = LFC_df.index.map({val: i for i, val in enumerate(exp_ids)})
LFC_df = LFC_df.sort_values("sort_col")
LFC_df = LFC_df.drop("sort_col", axis = 1)

# transpose
LFC_df = LFC_df.T

display(LFC_df)

CPC006_THP1_6H:C04  CPC006_THP1_6H:G06  CPC006_THP1_6H:K09  \
AARS             -0.387213           -0.196513            0.390337   
ABCB6            -0.326253            0.598648            0.330848   
ABCC5            -0.372873           -0.817998           -0.027023   
ABCF1             0.940842           -0.074857            0.891892   
ABCF3            -0.306452            0.088499            1.271148   
...                    ...                 ...                 ...   
ZNF395            2.670836            0.505236           -0.189213   
ZNF451            1.408164            1.956289            0.094364   
ZNF586            0.332435            0.718335           -8.212865   
ZNF589            1.713703           -0.861997           -0.674397   
ZW10              2.306386           -2.660339            2.179686   

        CPC006_THP1_6H:M21  EMU001_THP1_6H:B19  EMU001_THP1_3H:A22  \
AARS              0.711488           -1.631878           -3.350361   
ABCB6            -0.324052           -1.532262           -1.169512   
ABCC5             1.977602           -0.509829            0.294921   
ABCF1            -0.028908           -1.682907           -2.556273   
ABCF3            -0.294476           -2.347471           -2.476405   
...                    ...                 ...                 ...   
ZNF395            2.495137           -0.965475           -2.726208   
ZNF451            1.167063            0.434050           -0.673208   
ZNF586           -1.286765            0.744662            2.191138   
ZNF589            1.298077           -2.484406           -2.410790   
ZW10              0.206235            1.383043            0.918944   

        EMU001_THP1_24H:B19  EMU001_THP1_6H:L19  EMU001_THP1_6H:L24  \
AARS               0.227556           -1.951610           -2.608110   
ABCB6              0.192888           -1.123496           -1.262929   
ABCC5             -0.151395            0.176938            0.006621   
ABCF1             -0.051639           -1.539789           -1.756840   
ABCF3              0.325646           -1.496271           -2.029871   
...                     ...                 ...                 ...   
ZNF395             0.046509           -1.331926           -1.038859   
ZNF451             0.227517            0.620250            0.582883   
ZNF586            -0.044121            0.957162            1.875146   
ZNF589            -0.058223           -1.923722           -2.212456   
ZW10              -0.453207            0.265577            1.105044   

        EMU001_THP1_24H:L24  ...  AICHI001_THP1_4H:E19  AICHI001_THP1_4H:M15  \
AARS              -0.094777  ...             -1.022569             -1.559520   
ABCB6             -0.037912  ...             -0.220063             -0.325463   
ABCC5              1.474204  ...             -0.033072              0.268361   
ABCF1             -0.179273  ...             -0.479926             -0.972592   
ABCF3              0.351997  ...             -0.424225             -0.588116   
...                     ...  ...                   ...                   ...   
ZNF395            -0.469308  ...             -1.725468              0.093173   
ZNF451             0.515117  ...              1.529267              0.564175   
ZNF586            -0.279687  ...              1.483469              0.871011   
ZNF589            -0.109756  ...             -2.818812             -1.760613   
ZW10               0.486243  ...             -0.458875             -1.146250   

        AICHI002_THP1_24H:M09  AICHI001_THP1_24H:E22  AICHI001_THP1_24H:E23  \
AARS                -3.993821              -0.240303              -0.219036   
ABCB6               -0.933889              -0.119696              -0.131829   
ABCC5                3.964201              -0.108097              -0.084881   
ABCF1                4.273963              -0.482709               0.043341   
ABCF3               -3.544892              -0.119166               0.394718   
...                       ...         

In [13]:
# now we need to leave only siginfo data present in LFC df
sig_info_df = sig_info_df.loc[LFC_df.columns]
n_experiments = len(sig_info_df.index)
exp_ids = sig_info_df.index.unique().tolist()
print(n_experiments)
sig_info_df


263


bead_batch  nearest_dose  pert_dose pert_dose_unit  \
CPC006_THP1_6H:C04          f2b4         10.00  10.000000             uM   
CPC006_THP1_6H:G06          f2b4         10.00  10.000000             uM   
CPC006_THP1_6H:K09          f2b4         10.00  10.000000             uM   
CPC006_THP1_6H:M21          f2b4         10.00  10.000000             uM   
EMU001_THP1_6H:B19           b39         10.00  10.000000             uM   
...                          ...           ...        ...            ...   
AICHI001_THP1_4H:O14         b39          2.50   2.500000             uM   
AICHI001_THP1_24H:M18        b32          0.01   0.009766             uM   
AICHI001_THP1_4H:M13         b39         10.00  10.000000             uM   
AICHI001_THP1_4H:E24         b39          0.01   0.009766             uM   
AICHI001_THP1_4H:O17         b39          0.04   0.039062             uM   

                      pert_idose pert_itime  pert_time pert_time_unit  \
CPC006_THP1_6H:C04         10 uM        6 h        6.0              h   
CPC006_THP1_6H:G06         10 uM        6 h        6.0              h   
CPC006_THP1_6H:K09         10 uM        6 h        6.0              h   
CPC006_THP1_6H:M21         10 uM        6 h        6.0              h   
EMU001_THP1_6H:B19         10 uM        6 h        6.0              h   
...                          ...        ...        ...            ...   
AICHI001_THP1_4H:O14      2.5 uM        4 h        4.0              h   
AICHI001_THP1_24H:M18    0.01 uM       24 h       24.0              h   
AICHI001_THP1_4H:M13       10 uM        4 h        4.0              h   
AICHI001_THP1_4H:E24     0.01 uM        4 h        4.0              h   
AICHI001_THP1_4H:O17     0.04 uM        4 h        4.0              h   

                      cell_mfc_name             pert_mfc_id  ...  cell_iname  \
CPC006_THP1_6H:C04             THP1  BRD-K69932463-001-03-1  ...        THP1   
CPC006_THP1_6H:G06             THP1  BRD-K67566344-001-01-8  ...        THP1   
CPC006_THP1_6H:K09             THP1  BRD-K51575138-001-01-7  ...        THP1   
CPC006_THP1_6H:M21             THP1  BRD-A62025033-001-01-8  ...        THP1   
EMU001_THP1_6H:B19             THP1           BRD-A50998626  ...        THP1   
...                             ...                     ...  ...         ...   
AICHI001_THP1_4H:O14           THP1           BRD-K13566078  ...        THP1   
AICHI001_THP1_24H:M18          THP1           BRD-K05804044  ...        THP1   
AICHI001_THP1_4H:M13           THP1           BRD-K05804044  ...        THP1   
AICHI001_THP1_4H:E24           THP1           BRD-K53581288  ...        THP1   
AICHI001_THP1_4H:O17           THP1           BRD-K13566078  ...        THP1   

                       det_wells  \
CPC006_THP1_6H:C04           C04   
CPC006_THP1_6H:G06           G06   
CPC006_THP1_6H:K09           K09   
CPC006_THP1_6H:M21           M21   
EMU001_THP1_6H:B19           B19   
...                          ...   
AICHI001_THP1_4H:O14         O14   
AICHI001_THP1_24H:M18        M18   
AICHI001_THP1_4H:M13         M13   
AICHI001_THP1_4H:E24         E24   
AICHI001_THP1_4H:O17         O17   

                                                              det_plates  \
CPC006_THP1_6H:C04                    CPC006_THP1_6H_X1_F2B4_DUO52HI53LO   
CPC006_THP1_6H:G06     CPC006_THP1_6H_X1_F2B4_DUO52HI53LO|CPC006_THP1...   
CPC006_THP1_6H:K09                    CPC006_THP1_6H_X1_F2B4_DUO52HI53LO   
CPC006_THP1_6H:M21     CPC006_THP1_6H_X1_F2B4_DUO52HI53LO|CPC006_THP1...   
EMU001_THP1_6H:B19     EMU001_THP1_6H_X1_B39|EMU001_THP1_6H_X2_B39|EM...   
...                                                                  ...   
AICHI001_THP1_4H:O14     AICHI001_THP1_4H_X1_B39|AICHI001_THP1_4H_X3_B39   
AICHI001_THP1_24H:M18  AICHI001_THP1_24H_X1_B32|AICHI001_THP1_24H_X2_...   
AICHI001_THP1_4H:M13   AICHI001_THP1_4H_X1_B39|AICHI001_THP1_4H_X2_B3...   
AICHI001_THP1_4H:E24   AICHI001_THP1_4H_X1_B39|AICHI001_THP1_4H_X2_B3...   
AI

In [14]:
genes = LFC_df.index.tolist()
print('Landmark genes list: ', len(genes), genes)

n_genes = len(genes)

Landmark genes list:  978 ['AARS', 'ABCB6', 'ABCC5', 'ABCF1', 'ABCF3', 'ABHD4', 'ABHD6', 'ABL1', 'ACAA1', 'ACAT2', 'ACBD3', 'ACD', 'ACLY', 'ACOT9', 'ADAM10', 'ADAT1', 'ADGRE5', 'ADGRG1', 'ADH5', 'ADI1', 'ADO', 'ADRB2', 'AGL', 'AKAP8', 'AKAP8L', 'AKR7A2', 'AKT1', 'ALAS1', 'ALDH7A1', 'ALDOA', 'ALDOC', 'AMDHD2', 'ANKRD10', 'ANO10', 'ANXA7', 'APBB2', 'APOE', 'APP', 'APPBP2', 'ARFIP2', 'ARHGAP1', 'ARHGEF12', 'ARHGEF2', 'ARID4B', 'ARID5B', 'ARL4C', 'ARNT2', 'ARPP19', 'ASAH1', 'ASCC3', 'ATF1', 'ATF5', 'ATF6', 'ATG3', 'ATMIN', 'ATP11B', 'ATP1B1', 'ATP2C1', 'ATP6V0B', 'ATP6V1D', 'AURKA', 'AURKB', 'AXIN1', 'B4GAT1', 'BACE2', 'BAD', 'BAG3', 'BAMBI', 'BAX', 'BCL2', 'BCL7B', 'BDH1', 'BECN1', 'BHLHE40', 'BID', 'BIRC2', 'BIRC5', 'BLCAP', 'BLMH', 'BLVRA', 'BMP4', 'BNIP3', 'BNIP3L', 'BPHL', 'BRCA1', 'BTK', 'BUB1B', 'BZW2', 'C2CD2', 'C2CD2L', 'C2CD5', 'C5', 'CAB39', 'CALM3', 'CALU', 'CAMSAP2', 'CANT1', 'CAPN1', 'CARMIL1', 'CASC3', 'CASK', 'CASP10', 'CASP2', 'CASP3', 'CASP7', 'CAST', 'CAT', 'CBLB', 'CBR1

## Inhibitor concentrations, IC50, and perturbation matrices

In [15]:
inhib_conc_matrix = np.zeros((n_modules, n_experiments))
ic50_matrix = np.ones((n_modules, n_experiments))
gamma_matrix = np.zeros((n_modules, n_experiments))

In [16]:

for i, module in enumerate(modules):
    drugs_for_module = IC50_df.Drug[IC50_df.Module == module].tolist()
    for drug in drugs_for_module:
        # get IC50 for this drug
        ic50 = IC50_df.IC50[IC50_df.Drug == drug].values
        gamma = IC50_df.Gamma[IC50_df.Drug == drug].values
        #print(drug, ic50)
        assert ic50.size == 1
        assert gamma.size == 1
        # get experiments with this drug
        exp_with_drug = sig_info_df.index[sig_info_df.cmap_name == drug].tolist()
        #print(exp_with_drug) 
        for exp_id in exp_with_drug:
            j = exp_ids.index(exp_id)
            #print(j)
            # extract inhibitor concentration
            inhib_conc = sig_info_df.pert_dose[sig_info_df.index == exp_id].values
            assert inhib_conc.size == 1
            # insert values in matrices
            inhib_conc_matrix[i, j] = inhib_conc.item()
            ic50_matrix[i, j] = ic50.item()
            gamma_matrix[i, j] = gamma.item()


In [17]:
# transform matrices into pandas dfs for export with row and column names
inhib_conc_df = pd.DataFrame(inhib_conc_matrix, index = modules, columns = exp_ids)
ic50_df = pd.DataFrame(ic50_matrix, index = modules, columns = exp_ids)
gamma_df = pd.DataFrame(gamma_matrix, index = modules, columns = exp_ids)

# create binary perturbation matrix
pert_df = pd.DataFrame(
    np.where(inhib_conc_matrix != 0, 1, 0),
    index = inhib_conc_df.index,
    columns = inhib_conc_df.columns,
)

In [18]:
display(ic50_df)
display(gamma_df)
display(inhib_conc_df)
display(pert_df)

CPC006_THP1_6H:C04  CPC006_THP1_6H:G06  CPC006_THP1_6H:K09  \
CDK1/2                  1.00                1.00                 1.0   
CDK4/6                  1.00                1.00                 1.0   
IKK                     1.00                1.00                 0.1   
TLR4                    1.00                1.00                 1.0   
mTOR                    0.02                0.03                 1.0   
PAK                     1.00                1.00                 1.0   
ERK                     1.00                1.00                 1.0   
JAK/STAT                1.00                1.00                 1.0   
PLK1                    1.00                1.00                 1.0   
ITK                     1.00                1.00                 1.0   

          CPC006_THP1_6H:M21  EMU001_THP1_6H:B19  EMU001_THP1_3H:A22  \
CDK1/2                  1.00                1.00               1.000   
CDK4/6                  1.00                1.00               1.000   
IKK                     1.00                1.00               1.000   
TLR4                    1.00                1.00               0.002   
mTOR                    1.76                0.05               1.000   
PAK                     1.00                1.00               1.000   
ERK                     1.00                1.00               1.000   
JAK/STAT                1.00                1.00               1.000   
PLK1                    1.00                1.00               1.000   
ITK                     1.00                1.00               1.000   

          EMU001_THP1_24H:B19  EMU001_THP1_6H:L19  EMU001_THP1_6H:L24  \
CDK1/2                   1.00                1.00                1.00   
CDK4/6                   1.00                1.00                1.00   
IKK                      1.00                1.00                1.00   
TLR4                     1.00                1.00                1.00   
mTOR                     0.05                1.00                1.00   
PAK                      1.00                1.00                1.00   
ERK                      1.00                1.00                1.00   
JAK/STAT                 1.00                0.07                0.07   
PLK1                     1.00                1.00                1.00   
ITK                      1.00                1.00                1.00   

          EMU001_THP1_24H:L24  ...  AICHI001_THP1_4H:E19  \
CDK1/2                   1.00  ...                  1.00   
CDK4/6                   1.00  ...                  1.00   
IKK                      1.00  ...                  1.00   
TLR4                     1.00  ...                  1.00   
mTOR                     1.00  ...                  1.00   
PAK                      1.00  ...                  1.00   
ERK                      1.00  ...                  1.00   
JAK/STAT                 0.07  ...                  0.05   
PLK1                     1.00  ...                  1.00   
ITK                      1.00  ...                  1.00   

          AICHI001_THP1_4H:M15  AICHI002_THP1_24H:M09  AICHI001_THP1_24H:E22  \
CDK1/2                     1.0                  0.004                   1.00   
CDK4/6                     1.0                  1.000                   1.00   
IKK                        1.0                  1.000                   1.00   
TLR4                       1.0                  1.000                   1.00   
mTOR                       1.0                  1.000                   1.00   
PAK                        1.0                  1.000                   1.00   
ERK                        3.0                  1.000                   1.00   
JAK/STAT                   1.0                  1.000                   0.05   
PLK1                       1.0                  1.000                   1.00   
ITK                        1.0                  1.000                   1.00   

          AICHI001_THP1_24H:E23  AICHI001_THP1_4H:O14  AICHI001_THP1_24H:M18  

CPC006_THP1_6H:C04  CPC006_THP1_6H:G06  CPC006_THP1_6H:K09  \
CDK1/2                   0.0                 0.0                 0.0   
CDK4/6                   0.0                 0.0                 0.0   
IKK                      0.0                 0.0                 0.0   
TLR4                     0.0                 0.0                 0.0   
mTOR                     0.0                 0.0                 0.0   
PAK                      0.0                 0.0                 0.0   
ERK                      0.0                 0.0                 0.0   
JAK/STAT                 0.0                 0.0                 0.0   
PLK1                     0.0                 0.0                 0.0   
ITK                      0.0                 0.0                 0.0   

          CPC006_THP1_6H:M21  EMU001_THP1_6H:B19  EMU001_THP1_3H:A22  \
CDK1/2                   0.0                 0.0                 0.0   
CDK4/6                   0.0                 0.0                 0.0   
IKK                      0.0                 0.0                 0.0   
TLR4                     0.0                 0.0                 0.0   
mTOR                     0.0                 0.0                 0.0   
PAK                      0.0                 0.0                 0.0   
ERK                      0.0                 0.0                 0.0   
JAK/STAT                 0.0                 0.0                 0.0   
PLK1                     0.0                 0.0                 0.0   
ITK                      0.0                 0.0                 0.0   

          EMU001_THP1_24H:B19  EMU001_THP1_6H:L19  EMU001_THP1_6H:L24  \
CDK1/2                    0.0                 0.0                 0.0   
CDK4/6                    0.0                 0.0                 0.0   
IKK                       0.0                 0.0                 0.0   
TLR4                      0.0                 0.0                 0.0   
mTOR                      0.0                 0.0                 0.0   
PAK                       0.0                 0.0                 0.0   
ERK                       0.0                 0.0                 0.0   
JAK/STAT                  0.0                 0.0                 0.0   
PLK1                      0.0                 0.0                 0.0   
ITK                       0.0                 0.0                 0.0   

          EMU001_THP1_24H:L24  ...  AICHI001_THP1_4H:E19  \
CDK1/2                    0.0  ...                   0.0   
CDK4/6                    0.0  ...                   0.0   
IKK                       0.0  ...                   0.0   
TLR4                      0.0  ...                   0.0   
mTOR                      0.0  ...                   0.0   
PAK                       0.0  ...                   0.0   
ERK                       0.0  ...                   0.0   
JAK/STAT                  0.0  ...                   0.0   
PLK1                      0.0  ...                   0.0   
ITK                       0.0  ...                   0.0   

          AICHI001_THP1_4H:M15  AICHI002_THP1_24H:M09  AICHI001_THP1_24H:E22  \
CDK1/2                     0.0                    0.0                    0.0   
CDK4/6                     0.0                    0.0                    0.0   
IKK                        0.0                    0.0                    0.0   
TLR4                       0.0                    0.0                    0.0   
mTOR                       0.0                    0.0                    0.0   
PAK                        0.0                    0.0                    0.0   
ERK                        0.0                    0.0                    0.0   
JAK/STAT                   0.0                    0.0                    0.0   
PLK1                       0.0                    0.0                    0.0   
ITK                        0.0                    0.0                    0.0   

          AICHI001_THP1_24H:E23  AICHI001_THP1_4H:O14  AICHI001_THP1_24H:M18  

CPC006_THP1_6H:C04  CPC006_THP1_6H:G06  CPC006_THP1_6H:K09  \
CDK1/2                   0.0                 0.0                 0.0   
CDK4/6                   0.0                 0.0                 0.0   
IKK                      0.0                 0.0                10.0   
TLR4                     0.0                 0.0                 0.0   
mTOR                    10.0                10.0                 0.0   
PAK                      0.0                 0.0                 0.0   
ERK                      0.0                 0.0                 0.0   
JAK/STAT                 0.0                 0.0                 0.0   
PLK1                     0.0                 0.0                 0.0   
ITK                      0.0                 0.0                 0.0   

          CPC006_THP1_6H:M21  EMU001_THP1_6H:B19  EMU001_THP1_3H:A22  \
CDK1/2                   0.0                 0.0             0.00000   
CDK4/6                   0.0                 0.0             0.00000   
IKK                      0.0                 0.0             0.00000   
TLR4                     0.0                 0.0             0.15625   
mTOR                    10.0                10.0             0.00000   
PAK                      0.0                 0.0             0.00000   
ERK                      0.0                 0.0             0.00000   
JAK/STAT                 0.0                 0.0             0.00000   
PLK1                     0.0                 0.0             0.00000   
ITK                      0.0                 0.0             0.00000   

          EMU001_THP1_24H:B19  EMU001_THP1_6H:L19  EMU001_THP1_6H:L24  \
CDK1/2                    0.0                 0.0            0.000000   
CDK4/6                    0.0                 0.0            0.000000   
IKK                       0.0                 0.0            0.000000   
TLR4                      0.0                 0.0            0.000000   
mTOR                     10.0                 0.0            0.000000   
PAK                       0.0                 0.0            0.000000   
ERK                       0.0                 0.0            0.000000   
JAK/STAT                  0.0                10.0            0.009766   
PLK1                      0.0                 0.0            0.000000   
ITK                       0.0                 0.0            0.000000   

          EMU001_THP1_24H:L24  ...  AICHI001_THP1_4H:E19  \
CDK1/2               0.000000  ...                   0.0   
CDK4/6               0.000000  ...                   0.0   
IKK                  0.000000  ...                   0.0   
TLR4                 0.000000  ...                   0.0   
mTOR                 0.000000  ...                   0.0   
PAK                  0.000000  ...                   0.0   
ERK                  0.000000  ...                   0.0   
JAK/STAT             0.009766  ...                  10.0   
PLK1                 0.000000  ...                   0.0   
ITK                  0.000000  ...                   0.0   

          AICHI001_THP1_4H:M15  AICHI002_THP1_24H:M09  AICHI001_THP1_24H:E22  \
CDK1/2                   0.000                  0.625                0.00000   
CDK4/6                   0.000                  0.000                0.00000   
IKK                      0.000                  0.000                0.00000   
TLR4                     0.000                  0.000                0.00000   
mTOR                     0.000                  0.000                0.00000   
PAK                      0.000                  0.000                0.00000   
ERK                      0.625                  0.000                0.00000   
JAK/STAT                 0.000                  0.000                0.15625   
PLK1                     0.000                  0.000                0.00000   
ITK                      0.000                  0.000                0.00000   

          AICHI001_THP1_24H:E23  AICHI001_THP1_4H:O14  AICHI001_THP1_24H:M18  

CPC006_THP1_6H:C04  CPC006_THP1_6H:G06  CPC006_THP1_6H:K09  \
CDK1/2                     0                   0                   0   
CDK4/6                     0                   0                   0   
IKK                        0                   0                   1   
TLR4                       0                   0                   0   
mTOR                       1                   1                   0   
PAK                        0                   0                   0   
ERK                        0                   0                   0   
JAK/STAT                   0                   0                   0   
PLK1                       0                   0                   0   
ITK                        0                   0                   0   

          CPC006_THP1_6H:M21  EMU001_THP1_6H:B19  EMU001_THP1_3H:A22  \
CDK1/2                     0                   0                   0   
CDK4/6                     0                   0                   0   
IKK                        0                   0                   0   
TLR4                       0                   0                   1   
mTOR                       1                   1                   0   
PAK                        0                   0                   0   
ERK                        0                   0                   0   
JAK/STAT                   0                   0                   0   
PLK1                       0                   0                   0   
ITK                        0                   0                   0   

          EMU001_THP1_24H:B19  EMU001_THP1_6H:L19  EMU001_THP1_6H:L24  \
CDK1/2                      0                   0                   0   
CDK4/6                      0                   0                   0   
IKK                         0                   0                   0   
TLR4                        0                   0                   0   
mTOR                        1                   0                   0   
PAK                         0                   0                   0   
ERK                         0                   0                   0   
JAK/STAT                    0                   1                   1   
PLK1                        0                   0                   0   
ITK                         0                   0                   0   

          EMU001_THP1_24H:L24  ...  AICHI001_THP1_4H:E19  \
CDK1/2                      0  ...                     0   
CDK4/6                      0  ...                     0   
IKK                         0  ...                     0   
TLR4                        0  ...                     0   
mTOR                        0  ...                     0   
PAK                         0  ...                     0   
ERK                         0  ...                     0   
JAK/STAT                    1  ...                     1   
PLK1                        0  ...                     0   
ITK                         0  ...                     0   

          AICHI001_THP1_4H:M15  AICHI002_THP1_24H:M09  AICHI001_THP1_24H:E22  \
CDK1/2                       0                      1                      0   
CDK4/6                       0                      0                      0   
IKK                          0                      0                      0   
TLR4                         0                      0                      0   
mTOR                         0                      0                      0   
PAK                          0                      0                      0   
ERK                          1                      0                      0   
JAK/STAT                     0                      0                      1   
PLK1                         0                      0                      0   
ITK                          0                      0                      0   

          AICHI001_THP1_24H:E23  AICHI001_THP1_4H:O14  AICHI001_THP1_24H:M18  

## Global responses for DPD modules

According to our discussion, $R$ for DPD vectors can not be calculated with the same formula as for pathway activities. Instead we are assuming:

\begin{equation}
R_{DPD},j = DPD = STV_{DPD} \cdot LFC_j
\end{equation}

In [19]:
# load STV data frame
STVs = pd.read_excel(os.path.join(info_dir, "ALL_DATA.xlsx"), sheet_name = "STVs", index_col = 0)
STV_df = pd.DataFrame(np.zeros((len(LFC_df.index),len(STVs.columns))),index=LFC_df.index,columns=STVs.columns)
STV_df.loc[STVs.index] = STVs

display(STV_df)


norm_vec_resist  norm_vec_TB  norm_vec_load  norm_vec_damage  \
AARS           0.003751     0.000000       0.000000         0.000000   
ABCB6          0.005285     0.000000       0.000000         0.000000   
ABCC5         -0.000536     0.012496       0.014916         0.005246   
ABCF1         -0.004149     0.000000       0.009495        -0.006757   
ABCF3         -0.011733     0.000000       0.000000         0.000000   
...                 ...          ...            ...              ...   
ZNF395         0.010331     0.000000       0.000000         0.000000   
ZNF451         0.002226     0.000000       0.000000         0.000000   
ZNF586        -0.011612     0.000000       0.000000         0.000000   
ZNF589         0.011885     0.000000       0.000000         0.000000   
ZW10           0.005060     0.000000       0.035811        -0.005202   

        norm_vec_microa  
AARS           0.000000  
ABCB6          0.000000  
ABCC5          0.018935  
ABCF1         -0.004911  
ABCF3          0.000000  
...                 ...  
ZNF395         0.000000  
ZNF451         0.000000  
ZNF586         0.000000  
ZNF589         0.000000  
ZW10           0.008795  

[978 rows x 5 columns]

In [20]:

# create empty DPD data frame
DPD_df = pd.DataFrame(
    np.zeros((len(LFC_df.columns), len(STVs.columns))),
    index=LFC_df.columns,
    columns=["DPD_resist","DPD_TB","DPD_load","DPD_damage","DPD_microa"],
)

# populate
for exp_id in DPD_df.index:
    for i in range(len(STV_df.columns)):
        DPD_df.loc[exp_id].iloc[i] = np.dot(LFC_df.T.loc[exp_id], STV_df.iloc[:, i])

display(DPD_df)

DPD_resist    DPD_TB  DPD_load  DPD_damage  DPD_microa
CPC006_THP1_6H:C04      -0.769247 -0.252601  1.196721   -0.373718    1.183066
CPC006_THP1_6H:G06      -1.433162 -0.077167  3.227575    2.459033    3.694395
CPC006_THP1_6H:K09      -1.055396  1.000931  0.972759    1.125405    1.259931
CPC006_THP1_6H:M21       0.476039 -0.190516  0.805359   -1.533741    0.224698
EMU001_THP1_6H:B19       2.809095 -2.082674 -0.349816   -0.944147    1.536528
...                           ...       ...       ...         ...         ...
AICHI001_THP1_4H:O14     0.805766 -0.051584 -0.489611    0.369096    1.447355
AICHI001_THP1_24H:M18    0.139720 -0.207295  0.234453   -0.420837    0.300335
AICHI001_THP1_4H:M13     0.025901  0.052178  0.515420    0.403796    2.089234
AICHI001_THP1_4H:E24     0.485682  0.663713  0.494942    0.622713    2.062756
AICHI001_THP1_4H:O17     0.884213  0.011954 -0.078714    0.134261    1.446358

[263 rows x 5 columns]

In [21]:
# transform to R global
R_global_DPD_df = DPD_df.T
display(R_global_DPD_df)

CPC006_THP1_6H:C04  CPC006_THP1_6H:G06  CPC006_THP1_6H:K09  \
DPD_resist           -0.769247           -1.433162           -1.055396   
DPD_TB               -0.252601           -0.077167            1.000931   
DPD_load              1.196721            3.227575            0.972759   
DPD_damage           -0.373718            2.459033            1.125405   
DPD_microa            1.183066            3.694395            1.259931   

            CPC006_THP1_6H:M21  EMU001_THP1_6H:B19  EMU001_THP1_3H:A22  \
DPD_resist            0.476039            2.809095            3.472484   
DPD_TB               -0.190516           -2.082674           -0.780699   
DPD_load              0.805359           -0.349816           -0.435418   
DPD_damage           -1.533741           -0.944147           -0.238261   
DPD_microa            0.224698            1.536528            1.295538   

            EMU001_THP1_24H:B19  EMU001_THP1_6H:L19  EMU001_THP1_6H:L24  \
DPD_resist            -0.492566            1.343949            2.049479   
DPD_TB                -0.232271           -0.399831           -0.961312   
DPD_load              -0.552939           -0.752625           -0.333279   
DPD_damage            -0.947662            0.617739            0.361276   
DPD_microa            -0.649277            0.631197            1.206122   

            EMU001_THP1_24H:L24  ...  AICHI001_THP1_4H:E19  \
DPD_resist            -0.537570  ...              0.449639   
DPD_TB                -0.356245  ...              0.197868   
DPD_load              -0.071860  ...              0.295300   
DPD_damage             0.141554  ...             -0.130563   
DPD_microa            -0.251971  ...              1.756929   

            AICHI001_THP1_4H:M15  AICHI002_THP1_24H:M09  \
DPD_resist              0.447949              -0.343226   
DPD_TB                  0.043230              -1.079648   
DPD_load                0.571335              -0.758256   
DPD_damage              0.616341              -2.427922   
DPD_microa              2.279128              -1.153240   

            AICHI001_THP1_24H:E22  AICHI001_THP1_24H:E23  \
DPD_resist              -0.106273               0.326936   
DPD_TB                  -0.138204              -0.046571   
DPD_load                -0.498665              -0.358668   
DPD_damage               0.000183              -0.163283   
DPD_microa              -0.216475              -0.372471   

            AICHI001_THP1_4H:O14  AICHI001_THP1_24H:M18  AICHI001_THP1_4H:M13  \
DPD_resist              0.805766               0.139720              0.025901   
DPD_TB                 -0.051584              -0.207295              0.052178   
DPD_load               -0.489611               0.234453              0.515420   
DPD_damage              0.369096              -0.420837              0.403796   
DPD_microa              1.447355               0.300335              2.089234   

            AICHI001_THP1_4H:E24  AICHI001_THP1_4H:O17  
DPD_resist              0.485682              0.884213  
DPD_TB                  0.663713              0.011954  
DPD_load                0.494942             -0.078714  
DPD_damage              0.622713              0.134261  
DPD_microa              2.062756              1.446358  

[5 rows x 263 columns]

## Save outputs

In [22]:
# save metadata as pickle
all_metadata = {
    "modules": modules,
    "n_modules": n_modules,
    "drugs": drugs,
    "n_drugs": n_drugs,
    "exp_ids": exp_ids,
    "n_experiments": n_experiments,
    "genes": genes,
    "n_genes": n_genes,
}

print(all_metadata)

with open(os.path.join(out_dir, "metadata.pickle"), "wb") as f:
    pickle.dump(all_metadata, f, protocol = pickle.HIGHEST_PROTOCOL)

{'modules': ['CDK1/2', 'CDK4/6', 'IKK', 'TLR4', 'mTOR', 'PAK', 'ERK', 'JAK/STAT', 'PLK1', 'ITK'], 'n_modules': 10, 'drugs': ['AZ-628', 'palbociclib', 'BMS-509744', 'PF-03758309', 'KU-0063794', 'CGP-60474', 'TAK-733', 'baricitinib', 'TPCA-1', 'RAF-265', 'AZD-8055', 'palomid-529', 'temsirolimus', 'PD-0325901', 'resatorvid', 'FR-180204', 'BMS-345541', 'dinaciclib', 'tofacitinib', 'volasertib'], 'n_drugs': 20, 'exp_ids': ['CPC006_THP1_6H:C04', 'CPC006_THP1_6H:G06', 'CPC006_THP1_6H:K09', 'CPC006_THP1_6H:M21', 'EMU001_THP1_6H:B19', 'EMU001_THP1_3H:A22', 'EMU001_THP1_24H:B19', 'EMU001_THP1_6H:L19', 'EMU001_THP1_6H:L24', 'EMU001_THP1_24H:L24', 'EMU001_THP1_6H:B24', 'EMU001_THP1_3H:A24', 'EMU001_THP1_24H:A23', 'EMU001_THP1_3H:B19', 'EMU001_THP1_3H:A19', 'EMU001_THP1_3H:B20', 'EMU001_THP1_6H:L23', 'EMU001_THP1_6H:B21', 'EMU001_THP1_3H:L22', 'EMU001_THP1_3H:L24', 'EMU001_THP1_3H:L20', 'EMU001_THP1_6H:A19', 'EMU001_THP1_6H:A21', 'EMU001_THP1_24H:B23', 'EMU001_THP1_24H:A24', 'EMU001_THP1_6H:L22', '

In [23]:
# save doses and perturbation matrix
inhib_conc_df.to_csv(os.path.join(out_dir, "inhib_conc_annotated.csv"))
ic50_df.to_csv(os.path.join(out_dir, "ic50_annotated.csv"))
gamma_df.to_csv(os.path.join(out_dir, "gamma_annotated.csv"))
pert_df.to_csv(os.path.join(out_dir, "pert_annotated.csv"))

In [24]:
# save log fold change L1000 data
LFC_df.to_csv(os.path.join(out_dir, "L1000_LFC_data.csv"))

In [25]:
# save R_global for DPDs
R_global_DPD_df.to_csv(os.path.join(out_dir, "R_global_DPDonly_annotated.csv"))